In [1]:
import sys
sys.path.insert(1, "/home/gemeinl/code/brainfeatures/")

In [2]:
import numpy as np
import time
import pandas as pd

In [3]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [4]:
from pyriemann.utils.mean import mean_covariance
from pyriemann.tangentspace import TangentSpace

In [5]:
from brainfeatures.data_set.tuh_abnormal import TuhAbnormal
from brainfeatures.utils.data_util import reject_windows_with_outliers, split_into_epochs

In [ ]:
ds = TuhAbnormal("/home/gemeinl/data/pre_100_Hz_without_rejecting/train/", n_recordings=200, extension=".h5")

In [ ]:
ds.load()

In [ ]:
len(ds)

In [ ]:
[n for n in ds.file_names if "00008184_s001_t001" in n]

In [ ]:
ds.file_names.index('/home/gemeinl/data/pre_100_Hz_without_rejecting/train/abnormal/01_tcp_ar/081/00008184/s001_2011_09_21/00008184_s001_t001.h5')

In [ ]:
from brainfeatures.decoding.decode import get_X_y

In [ ]:
X, y = get_X_y(ds)

In [ ]:
len(X), X[0].shape, len(y)

In [ ]:
epochs = [split_into_epochs(x.values, sfreq=100, epoch_duration_s=6) for x in X]
epochs = [epoch[reject_windows_with_outliers(epoch) == False] for epoch in epochs]

In [ ]:
len(epochs), epochs[0].shape

In [ ]:
from pyriemann.estimation import Covariances

In [ ]:
cov = Covariances()
covs = [cov.fit_transform(subject) for subject in epochs] 

In [ ]:
ds_eval = TuhAbnormal("/home/gemeinl/data/pre_100_Hz_without_rejecting/eval/", n_recordings=None, extension=".h5", subset="eval")

In [ ]:
ds_eval.load()

In [ ]:
len(ds_eval)

In [ ]:
X_eval, y_eval = get_X_y(ds_eval)

In [ ]:
epochs_eval = [split_into_epochs(x.values, sfreq=100, epoch_duration_s=6) for x in X_eval]
epochs_eval = [epoch[reject_windows_with_outliers(epoch) == False] for epoch in epochs_eval]

In [ ]:
len(epochs_eval), epochs_eval[0].shape

In [ ]:
cov = Covariances()
covs_eval = [cov.fit_transform(subject) for subject in epochs_eval] 

np.save("/home/gemeinl/data/covs/eval_covs.npy", covs_eval)

np.save("/home/gemeinl/data/covs/eval_pathology_labels.npy", y_eval)

__load saved covariance matrices and labels from npy file instead of recomputing__

In [ ]:
covs = np.load("/home/gemeinl/data/covs/train_covs_without_822.npy")

In [ ]:
y = np.load("/home/gemeinl/data/covs/train_pathology_labels_without_822.npy")

In [ ]:
len(covs), covs[0].shape, len(y)

remove broken recording 822, 00008184_s001_t001

mask = len(covs) * [1]

mask[822] = 0

covs = [cov for i, cov in enumerate(covs) if mask[i] == 1]

y = [y_ for i, y_ in enumerate(y) if mask[i] == 1]

len(covs), covs[0].shape, len(y)

check for non-spd matrices and remove them

In [65]:
def check_SPD(covs):
    good_ids = []
    for i,cov in enumerate(covs):
        try:
            np.linalg.cholesky(cov)
            good_ids.append(i)
        except:
            continue

    return good_ids

In [17]:
all_good_ids = [check_SPD(cov) for cov in covs]

In [18]:
len(all_good_ids)

2716

In [19]:
covs = [cov[all_good_ids[i]] for i, cov in enumerate(covs)]

In [20]:
len(covs), covs[0].shape, len(y)

(2716, (187, 21, 21), 2716)

n_recs = 500
covs = covs[:n_recs]
y = y[:n_recs]

len(covs), covs[0].shape, len(y)

mean_covs = CovsToMeanCov().fit_transform(covs)

np.save("/home/gemeinl/data/covs/mean_train_covs.npy", mean_covs)

__load saved mean covariance matrices and labels from npy file instead of recomputing__

In [6]:
covs = np.load("/home/gemeinl/data/covs/mean_train_covs.npy")

In [7]:
y = np.load("/home/gemeinl/data/covs/train_pathology_labels_without_822.npy")

In [8]:
covs.shape, y.shape

((2716, 21, 21), (2716,))

In [9]:
assert len(covs) == len(y)

In [10]:
class TimedTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, transformer):
        self.transformer=transformer
    
    def fit(self, X, y=None):
        t = time.time()
        return_value = self.transformer.fit(X=X, y=y)
        print("{} fitting time: {:.2f}s".format(self.transformer.__repr__, time.time() - t))
        return return_value
    
    def transform(self, X):
        t = time.time()
        return_value = self.transformer.transform(X=X)
        print("{} transforming time: {:.2f}s".format(self.transformer.__repr__, time.time() - t))
        return return_value

In [11]:
class CovsToMeanCov(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        mean_covs = np.array([mean_covariance(cov) for cov in X])
        return mean_covs

In [12]:
def mean_acc(pred_df):
    accs = []
    for i, d in pred_df.groupby("id"):
        accs.append((d.y_pred == d.y_true).mean())
    return np.mean(accs)

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
n_folds = 5
shuffle = False

In [15]:
kf = KFold(n_splits=n_folds, shuffle=shuffle)

In [16]:
n, n_ch, _ = covs.shape

In [17]:
kernel="rbf"
adaptation = False
for C in [10]:
    print("C", C)
    pipe = Pipeline([
        #("mean_covs", TimedTransformer(CovsToMeanCov())),
        ("ts", TangentSpace(tsupdate=adaptation, metric="riemann")),
        #("scale", StandardScaler()),
        ("clf", SVC(kernel=kernel, C=C, probability=True, gamma="auto"))
    ])

    train_preds, test_preds, train_probas, test_probas = pd.DataFrame(),pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for fold_i, (train_i, test_i) in enumerate(kf.split(range(len(covs)))):
        train_X = covs[train_i]
        test_X = covs[test_i]    
        train_y = y[train_i]    
        test_y = y[test_i]    

        pipe.fit(train_X, train_y)
        probas_train = pipe.predict_proba(train_X)
        preds_train = pipe.predict(train_X)
        
        column_names = ["id", "y_pred", "y_true"]
        tmp = pd.DataFrame([len(train_y) * [fold_i], probas_train[:, 1], train_y], index=column_names).T
        train_probas = train_probas.append(tmp, ignore_index=True)  
        
        tmp2 = pd.DataFrame([len(train_y) * [fold_i], preds_train, train_y], index=column_names).T
        train_preds = train_preds.append(tmp2, ignore_index=True)  
        
        probas_test = pipe.predict_proba(test_X)
        preds_test = pipe.predict(test_X)

        tmp = pd.DataFrame([len(test_y) * [fold_i], probas_test[:, 1], test_y], index=column_names).T
        test_probas = test_probas.append(tmp, ignore_index=True)  
        
        tmp2 = pd.DataFrame([len(test_y) * [fold_i], preds_test, test_y], index=column_names).T
        test_preds = test_preds.append(tmp2, ignore_index=True)  
        
        print(fold_i, )

C 10


In [18]:
mean_acc(train_preds)

0.8692932431252803

In [19]:
mean_acc(test_preds)

0.8203255335283284

train_preds.to_csv("/home/gemeinl/results/all_recs_100_hz/features/riemannian/cv/predictions_train.csv")
test_preds.to_csv("/home/gemeinl/results/all_recs_100_hz/features/riemannian/cv/predictions_valid.csv")

train_probas.to_csv("/home/gemeinl/results/all_recs_100_hz/features/riemannian/pathological/cv/predictions_train.csv")
test_probas.to_csv("/home/gemeinl/results/all_recs_100_hz/features/riemannian/pathological/cv/predictions_valid.csv")

In [31]:
import json

In [32]:
config = {"C": C, "kernel": kernel, "adaptation": adaptation, "gamma": "auto", "scaling": None, "model": "Riemannian", "n_features":21*21, "n_runs": n_folds,
     "sfreq": 100, "shuffle": False}
with open("/home/gemeinl/results/all_recs_100_hz/features/riemannian/pathological/cv/config.json", "w") as json_file:
    json.dump(config, json_file, indent=4)

__run final evaluation__

In [35]:
mean_covs_eval = np.load("/home/gemeinl/data/covs/mean_eval_covs.npy")

In [36]:
y_eval = np.load("/home/gemeinl/data/covs/eval_pathology_labels.npy")

In [37]:
mean_covs_eval.shape, len(y_eval)

((276, 21, 21), 276)

In [38]:
pipe = Pipeline([
    ("ts", TimedTransformer(TangentSpace(tsupdate=adaptation, metric="riemann"))),
    ("clf", SVC(kernel=kernel, C=C, probability=True, gamma="auto"))
])

In [39]:
pipe.fit(covs, y)
preds = pipe.predict(mean_covs_eval)
preds_proba = pipe.predict_proba(mean_covs_eval)
train_preds = pipe.predict(covs)
train_proba = pipe.predict_proba(covs)

<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> fitting time: 12.71s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.08s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.08s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.98s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.66s


In [69]:
config = {"C": C, "kernel": kernel, "adaptation": adaptation, "gamma": "auto", "scaling": None, "model": "Riemannian", "n_features":21*21, "n_runs": 1,
     "sfreq": 100, "shuffle": False}
with open("/home/gemeinl/results/all_recs_100_hz/features/riemannian/pathological/eval/config.json", "w") as json_file:
    json.dump(config, json_file, indent=4)

In [64]:
df_train = pd.DataFrame([train_proba[:, 1], y, len(y) * [0]], index=["y_pred", "y_true", "id"]).T

In [65]:
((df_train.y_pred >= .5) == df_train.y_true).mean()

0.8696612665684831

In [66]:
df_eval = pd.DataFrame([preds_proba[:, 1], y_eval, len(y_eval) * [0]], index=["y_pred", "y_true", "id"]).T

In [67]:
((df_eval.y_pred >= .5) == df_eval.y_true).mean()

0.8586956521739131

df_train.to_csv("/home/gemeinl/results/all_recs_100_hz/features/riemannian/pathological/eval/predictions_train.csv")
df_eval.to_csv("/home/gemeinl/results/all_recs_100_hz/features/riemannian/pathological/eval/predictions_eval.csv")

In [19]:
train_preds = pd.read_csv("../results/features/riemannian/pathological/cv/predictions_train.csv", index_col=0)

In [24]:
for n, g in train_preds.groupby("id"):
    print(((g.y_pred >= .5) == g.y_true).mean())

0.8697053406998159
0.8729866543948458
0.8729866543948458
0.8725264611136677
0.868844914864243


In [27]:
test_preds = pd.read_csv("../results/features/riemannian/pathological/cv/predictions_valid.csv", index_col=0)

In [28]:
for n, g in test_preds.groupby("id"):
    print(((g.y_pred >= .5) == g.y_true).mean())

0.8106617647058824
0.8139963167587477
0.8139963167587477
0.8139963167587477
0.8103130755064457


In [29]:
np.mean([0.8106617647058824,
0.8139963167587477,
0.8139963167587477,
0.8139963167587477,
0.8103130755064457
])

0.8125927580977143

In [30]:
eval_preds = pd.read_csv("../results/features/riemannian/pathological/eval/predictions_eval.csv", index_col=0)

In [31]:
for n, g in eval_preds.groupby("id"):
    print(((g.y_pred >= .5) == g.y_true).mean())

0.8586956521739131
